# Newton-Raphson method

## Reading

* [The source of the technique discussion in class](https://web.mit.edu/6.001/6.037/sicp.pdf): see pages 28-31 from the *Structure and Interpretation of Computer Programs* (SICP). The code is written in LISP, which is explained in pages 1-28. SICP is among **my [top-10 books](https://lgreco.github.io/cdp/Top10.html)** in computer science.

* [The Method of Fluxions](https://archive.org/details/methodoffluxions00newt/page/n3/mode/2up) by Isaac Newton. This is the beginning of calculus and untold suffering of millions of college students since. It is listed here for its historic value.

* [A biography of Gottfried Wilhelm Leibnitz](https://etc.usf.edu/lit2go/218/a-short-account-of-the-history-of-mathematics/5539/gottfried-wilhelm-leibnitz/) including a description of the calculus controversy -- over who had first invented calculus: Newton or Leibnitz.

* Peter Deuflhard's [brief history of Newton's method](https://ems.press/content/book-chapter-files/27348).

* A more extensive [historical review by Tjalling J Ypma](./NewtonRaphson_SIAMRev.pdf)

---

The Newton-Raphson (or Newtons chord method, or just Newton's method) is a numerical approximation to the root of a function, $f(x) = 0$.

The approximation is given by

$$
x_{n+1} = x_n - \frac{f(x_n)}{f'(x_n)}
$$

with $x_0$ some realistic guess about the root of the function $f(x)$.

Suppose we with to compute $\sqrt z$ for some real number $z$. If $x=\sqrt z$ we can write:

$$x^2 = z\Rightarrow x^2-z=0$$

If we set $f(x)=x^2-z$ we can use Newton's method to solve for $f(x)=0$. Noting that $f'(x)=2x$ we can approximate the root of the function as:

$$
\begin{align*}
x_{n+1} & = x_n - \frac{f(x_n)}{f'(x_n)}
        && (\text{Definition of Newton's approximation}) \\
        & = x_n - \frac{x_n^2-z}{2x_n}
        && (\text{Substitute}\ f=x^2-z; f'=2x) \\
        & = \frac{2x^2_n-(x_n^2-z)}{2x_n}
        && (\text{Merge fractions})  \\
        & = \frac{x^2_n+z}{2x_n}
        && (\text{Simplify numerator}) \\
        & = \frac{x^2_n}{2x_n} + \frac{z}{2x_n}
        && (\text{Separate fractions}) \\
        & = \frac{x_n}{2} + \frac{z}{2x_n}
        && (\text{Simplify first fraction})  \\
        & = \frac{1}{2} \left({x_n} + \frac{z}{x_n}\right)
        && (\text{Factor out}\ \tfrac{1}{2})  \\
\end{align*}
$$

The expression above tells us that the improved guess $x_{n+1}$ is the average of $x_n$ and ${z}/{x_n}$.

With this information we can write a simple algorithm to compute the square root of a number.

$$
\begin{align*}
& \textbf{square root of } (z) \\
& \qquad \text{guess} = 1 \\
& \qquad \textbf{if}\ \text{guess is good enough} \\
& \qquad\qquad \textbf{return}\ \text{guess} \\
& \qquad \textbf{else} \\
& \qquad\qquad \textbf{improve}\ \text{guess and try again} \\
\end{align*}
$$

The success of the algorithm above depends on how to assess if a guess is good enough and how to improve it if it's not. An improved guess, as we saw already, is the average $(\text{guess}+z/\text{guess})/2$.

A guess is good enough when $\left|\text{guess}^2-z \right|\leq\epsilon$, with $\epsilon$ being the accuracy we desire for the computation.

We can now orchestrate everying in simple methods. To keep things tight, I prefer to organize these methods within an object.


In [ ]:
class SQRT:
    """Methods to compute the square root of a number using
    Newton's method."""

    # The maximum error allowed in the result.
    # The smaller the tolerance, the more accurate the result,
    # but the more iterations it will take to compute.
    _DEFAULT_TOLERANCE: float = 0.1

    def __init__(self, tolerance: float = _DEFAULT_TOLERANCE) -> None:
        """Initialize the object with the given tolerance."""
        # The value whose square root I am looking for.
        # I need to store it as an instance variable because
        # the methods that compute the square root need to access it.
        # The actual value will be set when the user-facing
        # sqrt method is called.
        self._z: float = 0
        # The tolerance for the approximation. This is a parameter that
        # controls the accuracy of the result.
        self._tolerance: float = tolerance

    def _average(self, a: float, b: float) -> float:
        """Return the average of a and b."""
        # The denominator below is not a magic value.
        # It is the number of values we are averaging.
        return (a + b) / 2.0

    def _good_enough(self, guess: float) -> bool:
        """Return True if guess is close enough to the square root of z."""
        return abs(guess * guess - self._z) <= self._tolerance

    def _improve(self, guess: float) -> float:
        """Return a better guess for the square root of z.
        The improvement is basically the average of the current
        guess and z divided by the guess."""
        return self._average(guess, self._z / guess)

    def _iter_sqrt(self, guess: float) -> float:
        """Return the square root of z using guess as a starting point."""
        if self._good_enough(guess):
            # If the guess is good enough, we can return it as the result.
            return guess  # that's the square root of z within tolerance
        else:
            # If the guess is not good enough, we need to improve it
            # and try again.
            return self._iter_sqrt(self._improve(guess))

    def sqrt(self, z: float) -> float:
        """User-facing method to compute the square root of z."""
        # Set the value of z as an instance variable so that the
        # other methods can access it.
        self._z = z
        # Start the iterative process with an initial guess of 1.0.
        return self._iter_sqrt(1.0)

In [ ]:
# Simple demonstration and test of the SQRT class.
if __name__ == "__main__":
    demo = SQRT()
    print("\033[2J", end="")  # Clear the screen (ANSI escape code)
    print("\nTest       Newton's         Actual")
    print("   x       sqrt(x)          sqrt(x)")
    print("-" * 40)
    for num in [4, 2, 10, 1234]:
        print(f"{num:-4d}      {demo.sqrt(num):9.6f}       {num**0.5:9.6f}")
    print("-" * 40, end="\n\n")

Class `SQRT` above actually implements a `while` loop without using the `while` statement. The collective behavior of the methods in `SQRT` is the equivalent of the following code.


In [ ]:
def good_enough(guess: float, z: float, tolerance: float) -> bool:
    return abs(guess * guess - z) <= tolerance


def average(a: float, b: float) -> float:
    return (a + b) / 2.0


def improve(guess: float, z: float) -> float:
    return average(guess, z / guess)


def iterative_sqrt(z: float, tolerance: float) -> float:
    current_guess: float = 1.0
    while not good_enough(current_guess, z, tolerance):
        current_guess = improve(current_guess, z)
    return current_guess

In [ ]:
# Simple demonstration and test of the the iterative method.
if __name__ == "__main__":
    demo = SQRT()
    print("\033[2J", end="")  # Clear the screen (ANSI escape code)
    print("\nTest       Newton's         Actual")
    print("   x       sqrt(x)          sqrt(x)")
    print("-" * 40)
    for num in [4, 2, 10, 1234]:
        print(f"{num:-4d}      {iterative_sqrt(num, 0.1):9.6f}       {num**0.5:9.6f}")
    print("-" * 40, end="\n\n")